# Goal: Fairness Metrics Computation

The goal of this notebook is to computes statistical fairness metrics, after the execution of a set of mitigation techniques, given:
- a training dataset (mitigated in case of a pre-processing technique),
- a target variable,
- a sensible attribute.


# Import Libraries





In [1]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  import sys
  path_to_project = '/content/drive/MyDrive/FairAlgorithm'
  sys.path.append(path_to_project)
  !sudo apt install libcairo2-dev pkg-config python3-dev
  IN_COLAB = True
except:
  IN_COLAB = False

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04.1).
python3-dev set to manually installed.
The following packages were automatically installed and are no longer required:
  libbz2-dev libpkgconf3 libreadline-dev
Use 'sudo apt autoremove' to remove them.
The following additional packages will be installed:
  libblkid-dev libblkid1 libcairo-script-interpreter2 libffi-dev
  libglib2.0-dev libglib2.0-dev-bin libice-dev liblzo2-2 libmount-dev
  libmount1 libpixman-1-dev libselinux1-dev libsepol-dev libsm-dev
  libxcb-render0-dev libxcb-shm0-dev
Suggested packages:
  libcairo2-doc libgirepository1.0-dev libglib2.0-doc libgdk-pixbuf2.0-bin
  | libgdk-pixbuf2.0-dev libxml2-utils libice-doc cryptsetup-bin libsm-doc
The following packages will be REMOVED:
  pkgconf r-base-dev
The following NEW packages will be installed:
  libblkid-dev libcairo-script-interpreter2 

In [2]:
#import libraries
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_validate,cross_val_score,cross_val_predict,train_test_split,StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm

# Configure the notebook


In the next code cell, set all the variables that are used throughout the notebook.  
The variables are used to configure the notebook, and to set the paths to the data files.

Modify the variables in the next code cell to configure the notebook

- `dataset_name`: The name of the dataset file.
- `dataset_path`: The path to the dataset file.
- `target`: The target feature to predict.
- `target_variable_labels`: The labels for the target feature.
- `sensible_attribute`: The sensible attribute to use for bias mitigation.

In [3]:
dataset_name = 'diabetes-women' #options: "diabetes-women", "sepsis", 'aids', "myocardial-infarction", 'alzheimer-disease', "diabetes-prediction", "stroke-prediction"

In [4]:
from source.utils.config import *
print(datasets_config)


dataset_config = datasets_config[dataset_name]
ignore_cols = dataset_config['ignore_cols']
target_variable = dataset_config['target_variable']
target_variable_labels = dataset_config['target_variable_labels']
sensible_attributes = dataset_config['sensible_attributes']
default_mappings = dataset_config.get('default_mappings', {})

{'diabetes-women': {'ignore_cols': ['Age'], 'target_variable': 'Outcome', 'target_variable_labels': [1, 0], 'sensible_attributes': ['AgeCategory'], 'default_mappings': {'label_maps': [{1.0: 'Diabetic', 0.0: 'NonDiabetic'}], 'protected_attribute_maps': [{1.0: 'Adult', 0.0: 'Young'}]}}, 'sepsis': {'ignore_cols': [], 'target_variable': 'Mortality', 'target_variable_labels': [1, 0], 'sensible_attributes': ['Gender_cat', 'Age_cat'], 'default_mappings': {}}, 'aids': {'ignore_cols': [], 'target_variable': 'cid', 'target_variable_labels': [1, 0], 'sensible_attributes': ['homo_cat', 'race_cat', 'age_cat'], 'default_mappings': {}}, 'myocardial-infarction': {'ignore_cols': [], 'target_variable': 'LET_IS_cat', 'target_variable_labels': [1, 0], 'sensible_attributes': ['SEX'], 'default_mappings': {'label_maps': [{1: 'Complication', 0: 'No Alzheimer'}], 'protected_attribute_maps': [{0: 'Female', 1: 'Male'}]}}, 'alzheimer-disease': {'ignore_cols': [], 'target_variable': 'Diagnosis', 'target_variable_l

# Libraries to compute performance and fairness metrics

We need to save the indexes of both groups `privileged` and `discriminated` in two lists.

`y_privileged` is the part the dataset where `sensible_value` = 1 (for example `AgeCategory` = 1), and `y_discriminated` is the part of dataset where `sensible_value` = 0.

Build the confusion matrices (one for the privileged group, one for the discriminated group) for each model.


Not for in-processing that has only one ML model!

In [5]:
def compute_scores(predictions_and_tests, models, n_splits):
  precision = {}
  recall = {}
  accuracy = {}
  f1_score = {}

  if mitigation not in without_model_mitigations:
    for model_name in (models):

      precisions = []
      recalls = []
      accuracys = []
      f1_scores = []
      for i in range(0,n_splits):
        y_test = predictions_and_tests[model_name][i]['y_test']
        y_pred = predictions_and_tests[model_name][i]['y_pred']
        #print(len(y_test), len(y_pred))
        precisions.append(metrics.precision_score(y_test, y_pred))
        recalls.append(metrics.recall_score(y_test, y_pred))
        accuracys.append(metrics.accuracy_score(y_test, y_pred))
        f1_scores.append(metrics.f1_score(y_test, y_pred))
      precision[model_name] = precisions
      recall[model_name] = recalls
      accuracy[model_name] = accuracys
      f1_score[model_name] = f1_scores
  else:
    precisions = []
    recalls = []
    accuracys = []
    f1_scores = []
    for i in range(0,n_splits):
        y_test = predictions_and_tests[i]['y_test']
        y_pred = predictions_and_tests[i]['y_pred']
        #print(len(y_test), len(y_pred))
        precisions.append(metrics.precision_score(y_test, y_pred))
        recalls.append(metrics.recall_score(y_test, y_pred))
        accuracys.append(metrics.accuracy_score(y_test, y_pred))
        f1_scores.append(metrics.f1_score(y_test, y_pred))
    precision = precisions
    recall = recalls
    accuracy = accuracys
    f1_score = f1_scores
  return accuracy, precision, recall, f1_score

In [6]:
def compute_mean_std_dev(metric_list, models):
  metric_dict = {}
  if models is not None:
    for model_name in (models):
      metric = np.array(metric_list[model_name])
      mean_metric = metric.mean()
      std_metric = metric.std()
      metric_dict[model_name] = [mean_metric, std_metric]
  else:
    metric = np.array(metric_list)
    mean_metric = metric.mean()
    std_metric = metric.std()
    metric_dict = [mean_metric, std_metric]
  return metric_dict

In [7]:
def compute_confusion_matrices(predictions_and_tests, target_variable_labels, models, n_splits):
  confusion_matrices = {}
  if mitigation not in without_model_mitigations:
    for model_name in (models):
      cm_splits = {}
      for i in range(0,n_splits):
        temp_dict = {}
        cm_priviliged = {}
        cm_discriminated = {}
        y_test = predictions_and_tests[model_name][i]['y_test']
        y_pred = predictions_and_tests[model_name][i]['y_pred']
        s_test = predictions_and_tests[model_name][i]['s_test']

        df_metrics = pd.DataFrame({'s_test': s_test, 'y_test':y_test, 'y_pred':y_pred})

        df_discrim = df_metrics[df_metrics['s_test'] == 0]
        #len_dicr = len(df_discrim)
        df_priv = df_metrics[df_metrics['s_test'] == 1]
        #len_priv = len(df_priv)

        cm_discriminated = confusion_matrix(df_discrim['y_test'], df_discrim['y_pred'], labels=target_variable_labels)
        cm_privileged = confusion_matrix(df_priv['y_test'], df_priv['y_pred'], labels=target_variable_labels)
        temp_dict['discriminated'] = cm_discriminated
        temp_dict['privileged'] = cm_privileged
        cm_splits[i] = temp_dict
      confusion_matrices[model_name] = cm_splits
  else:
    cm_splits = {}
    for i in range(0,n_splits):
      temp_dict = {}
      cm_priviliged = {}
      cm_discriminated = {}
      y_test = predictions_and_tests[i]['y_test']
      y_pred = predictions_and_tests[i]['y_pred']
      s_test = predictions_and_tests[i]['s_test']

      df_metrics = pd.DataFrame({'s_test': s_test, 'y_test':y_test, 'y_pred':y_pred})

      df_discrim = df_metrics[df_metrics['s_test'] == 0]
      #len_dicr = len(df_discrim)
      df_priv = df_metrics[df_metrics['s_test'] == 1]
      #len_priv = len(df_priv)

      cm_discriminated = confusion_matrix(df_discrim['y_test'], df_discrim['y_pred'], labels=target_variable_labels)
      cm_privileged = confusion_matrix(df_priv['y_test'], df_priv['y_pred'], labels=target_variable_labels)
      temp_dict['discriminated'] = cm_discriminated
      temp_dict['privileged'] = cm_privileged
      cm_splits[i] = temp_dict
      confusion_matrices = cm_splits
  return confusion_matrices

##Functions to compute fairness metrics

Terminology:

- d is the predicted value,
- Y is the actual value in the dataset
- G the protected attribute, priv= privileged group, discr=discriminated group
- L is the legittimate attribute (only for Conditional Statistical Parity)

Fairness Metrics List:

1. Group Fairness: (d=1|G=priv) = (d=1|G=discr)
2. Predictive Parity: (Y=1|d=1,G=priv) = (Y=1|d=1,G=discr)
3. Predictive Equality: (d=1|Y=0,G=priv) = (d=1|Y=0,G=discr)
4. Equal Opportunity:  (d=0|Y=1,G=priv) = (d=0|Y=1,G=discr)
5. Equalized Odds: (d=1|Y=i,G=priv) = (d=1|Y=i,G=discr), i ∈ 0,1
6. ConditionalUseAccuracyEquality: (Y=1|d=1, G=priv) = (Y=1|d=1,G=discr) and (Y=0|d=0,G=priv) = (Y=0|d=0,G=discr)
7. Overall Accuracy Equality: (d=Y, G=priv) = (d=Y, G=priv)
8. Treatment Equality: (Y=1, d=0, G=priv)/(Y=0, d=1, G=priv) = (Y=1, d=0, G=discr)/(Y=0, d=1, G=discr)
9. FOR Parity: (Y=1|d=0, G=priv) = (Y=1|d=0,G=discr)

How to evaluate the results?

Looking at the value for each corresponding metric:

- If the value is between 0 and 1-t the discriminated group suffers from unfairness
- If the value is greater than 1+t the privileged group suffers from unfairness
- If the value is between 1-t and 1+t both privileged and discriminated group have a fair treatment

t is a threshold that should be choose by the user according to the context and the goal of the task.


In [8]:
# Retrieve TP, TN, FP, FN values from a confusion matrix
def retrieve_values(cm):
  TN = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TP = cm[1][1]
  total = TN+FP+FN+TP
  return TP, TN, FP, FN, total

def rescale(metric):
  metric = metric - 1
  return metric

def standardization(metric):
  if metric > 1:
    metric = 1
  elif metric < -1:
    metric = -1
  return metric

def valid(metric, th):
  if metric > 1-th and metric < 1+th:
    return True
  return False

def and_function(m1, m2, th):
  if m1 > 1+th and m2 > 1+th:
    return max(m1, m2)
  elif m1 < 1-th and m2 < 1-th:
    return min(m1, m2)
  elif valid(m1, th) and valid(m2, th):
    return max(m1, m2)
  elif (valid(m1, th) or valid(m2, th)) and (m1 > 1+th or m2 > 1+th):
    return max(m1, m2)
  elif (valid(m1, th) or valid(m2, th)) and (m1 < 1-th or m2 < 1-th):
    return min(m1, m2)
  else:
    return max(m1, m2)

def sub_valid(metric, th):
  if metric > -th and metric < th:
    return True
  return False

def sub_and_function(m1, m2, th):
  if m1 >th and m2 > th:
    return max(m1, m2)
  elif m1 < -th and m2 < -th:
    return min(m1, m2)
  elif sub_valid(m1, th) and valid(m2, th):
    return max(m1, m2)
  elif (sub_valid(m1, th) or sub_valid(m2, th)) and (m1 > th or m2 > th):
    return max(m1, m2)
  elif (sub_valid(m1, th) or sub_valid(m2, th)) and (m1 < -th or m2 < -th):
    return min(m1, m2)
  else:
    return max(m1, m2)

In [9]:
# Fairness metrics computed using division operator
def fairness_metrics_division(confusion_matrix, threshold = 0.15):

  TP_priv, TN_priv, FP_priv, FN_priv, len_priv = retrieve_values(confusion_matrix['privileged'])
  TP_discr, TN_discr, FP_discr, FN_discr, len_discr = retrieve_values(confusion_matrix['discriminated'])

  GroupFairness_discr = (TP_discr+FP_discr)/len_discr
  GroupFairness_priv = (TP_priv+FP_priv)/len_priv
  if GroupFairness_priv == 0:
    GroupFairness = 2  #max value
  else:
     GroupFairness = GroupFairness_discr/GroupFairness_priv

  if TP_discr+FP_discr == 0:
    PredictiveParity_discr = 0
    PredictiveParity = 0  #min value
  else:
    PredictiveParity_discr = (TP_discr)/(TP_discr+FP_discr)
  if TP_priv+FP_priv == 0:
    PredictiveParity_priv = 0
    PredictiveParity = 2  #max value
  else:
    PredictiveParity_priv = (TP_priv)/(TP_priv+FP_priv)
  if PredictiveParity_discr != 0 and PredictiveParity_priv != 0:
    PredictiveParity = PredictiveParity_discr/PredictiveParity_priv
  elif PredictiveParity_priv == 0:
    PredictiveParity = 2  #max value
  else:
    PredictiveParity = 0  #min value

  if TN_discr+FP_discr == 0:
    PredictiveEquality_discr = 0
    PredictiveEquality = 0  #min value
  else:
    PredictiveEquality_discr = (FP_discr)/(TN_discr+FP_discr)
  if TN_priv+FP_priv == 0:
    PredictiveEquality_priv = 0
    PredictiveEquality = 2  #max value
  else:
    PredictiveEquality_priv = (FP_priv)/(TN_priv+FP_priv)
  if PredictiveEquality_discr != 0 and PredictiveEquality_priv != 0:
    PredictiveEquality = PredictiveEquality_discr/PredictiveEquality_priv
  elif PredictiveEquality_priv == 0:
    PredictiveEquality = 2  #max value
  else:
    PredictiveEquality = 0  #min value

  if FN_priv+TP_priv == 0:
    EqualOpportunity_priv = 0
    EqualOpportunity = 2  #max value
  else:
    EqualOpportunity_priv = (FN_priv)/(TP_priv+FN_priv)
  if FN_discr+TP_discr == 0:
    EqualOpportunity_discr = 0
    EqualOpportunity = 0  #min value
  else:
    EqualOpportunity_discr = (FN_discr)/(TP_discr+FN_discr)
  if EqualOpportunity_priv != 0 and EqualOpportunity_discr != 0:
    EqualOpportunity = EqualOpportunity_discr/EqualOpportunity_priv
  elif EqualOpportunity_priv == 0:
    EqualOpportunity = 2  #max value
  else:
    EqualOpportunity = 0  #min value

  if FN_discr+TP_discr == 0:
    EqualizedOdds1 = 0
    EqualizedOdds = 0 #min value
  elif FN_priv+TP_priv == 0:
    EqualizedOdds1 = 0
    EqualizedOdds = 2 #max value
  elif (TP_priv/(TP_priv+FN_priv)) == 0:
    EqualizedOdds1 = 2 #max value
  else:
    EqualizedOdds1 = ((TP_discr/(TP_discr+FN_discr)) / (TP_priv/(TP_priv+FN_priv))) # (1-equalOpportunity_discr)/(1-equalOpportunity_priv)
  if TN_priv+FP_priv == 0:
    EqualizedOdds2 = 0
    EqualizedOdds = 2 #max value
  elif TN_discr+FP_discr == 0:
    EqualizedOdds2 = 0
    EqualizedOdds = 0 #min value
  elif (FP_priv/(TN_priv+FP_priv)) == 0:
    EqualizedOdds2 = 2 #max value
  else:
    EqualizedOdds2 = ((FP_discr/(TN_discr+FP_discr)) / (FP_priv/(TN_priv+FP_priv))) # = PredictiveEquality
  if EqualizedOdds1 != 0 and EqualizedOdds2 != 0:
    EqualizedOdds = and_function(EqualizedOdds1, EqualizedOdds2, threshold)
  else:
    EqualizedOdds = 2 #max value

  if TP_discr+FP_discr == 0 or TN_discr+FP_discr == 0:
    ConditionalUseAccuracyEquality1 = 0
    ConditionalUseAccuracyEquality= 0 #min value
  elif (TP_priv/(TP_priv+FP_priv)) == 0:
    ConditionalUseAccuracyEquality1 = 2 #max value
  else:
    ConditionalUseAccuracyEquality1 = ((TP_discr/(TP_discr+FP_discr)) / (TP_priv/(TP_priv+FP_priv)))
  if TN_discr+FN_discr == 0 or TN_priv+FN_priv == 0:
    ConditionalUseAccuracyEquality2 = 0
    ConditionalUseAccuracyEquality = 2 #max value
  elif (TN_priv/(TN_priv+FN_priv)) == 0:
    ConditionalUseAccuracyEquality2 = 2 #max value
  else:
    ConditionalUseAccuracyEquality2 = ((TN_discr/(TN_discr+FN_discr)) / (TN_priv/(TN_priv+FN_priv)))
  if ConditionalUseAccuracyEquality1 != 0 and ConditionalUseAccuracyEquality2 != 0:
    ConditionalUseAccuracyEquality = and_function(ConditionalUseAccuracyEquality1, ConditionalUseAccuracyEquality2, threshold)
  else:
    ConditionalUseAccuracyEquality = 2 #max value

  OAE_discr = (TP_discr+TN_discr)/len_discr
  OAE_priv = (TP_priv+TN_priv)/len_priv
  if OAE_priv != 0 and OAE_priv != 0:
    OverallAccuracyEquality = OAE_discr/OAE_priv
  if OAE_priv == 0:
    OverallAccuracyEquality = 2  #max value
  else:
    OverallAccuracyEquality = 0 #min value

  if FP_priv == 0:
    TreatmentEquality_priv = 0
    TreatmentEquality = 2  #max value
  else:
    TreatmentEquality_priv = (FN_priv/FP_priv)
  if FP_discr == 0:
    TreatmentEquality_discr = 0
    TreatmentEquality = 0 #min value
  elif (FN_discr/FP_discr) == 0:
    TreatmentEquality_discr = 0 #max value
    TreatmentEquality = 0 #min value
  else:
    TreatmentEquality_discr = (FN_discr/FP_discr)
  if TreatmentEquality_priv != 0 and TreatmentEquality_discr != 0:
    TreatmentEquality = TreatmentEquality_discr/TreatmentEquality_priv
  elif TreatmentEquality_priv == 0:
    TreatmentEquality = 2 #max value
  else:
    TreatmentEquality = 0 #min value

  if TN_priv+FN_priv == 0:
    FORParity_priv = 0
    FORParity = 2 #max value
  else:
    FORParity_priv = (FN_priv)/(TN_priv+FN_priv)
  if TN_discr+FN_discr == 0:
    FORParity_discr = 0
    FORParity = 0  #min value
  elif (FN_discr)/(TN_discr+FN_discr) == 0:
    FORParity_discr = 0
    FORParity = 0 #min value
  else:
    FORParity_discr = (FN_discr)/(TN_discr+FN_discr)
  if FORParity_priv != 0 and FORParity_discr != 0:
    FORParity = FORParity_discr/FORParity_priv
  elif FORParity_priv == 0:
    FORParity = 2 #max value
  else:
    FORParity = 0 #min value


  FN_P_discr = (FN_discr)/len_discr
  FN_P_priv = (FN_priv)/len_priv
  if FN_P_priv == 0:
    FN_metric = 2  #max value
  else:
    FN_metric = FN_P_discr/FN_P_priv


  FP_P_discr = (FP_discr)/len_discr
  FP_P_priv = (FP_priv)/len_priv
  if FP_P_priv == 0:
    FP_metric = 2  #max value
  else:
    FP_metric = FP_P_discr/FP_P_priv


  #RecallParity = (TP_discr/(TP_discr+FN_discr))/(TP_priv/(TP_priv+FN_priv))

  metrics = {}
  metrics['GroupFairness'] = [GroupFairness, GroupFairness_discr, GroupFairness_priv]
  metrics['PredictiveParity'] = [PredictiveParity, PredictiveParity_discr, PredictiveParity_priv]
  metrics['PredictiveEquality'] = [PredictiveEquality, PredictiveEquality_discr, PredictiveEquality_priv]
  metrics['EqualOpportunity'] = [EqualOpportunity, EqualOpportunity_discr, EqualOpportunity_priv]
  metrics['EqualizedOdds'] = [EqualizedOdds, EqualizedOdds1, EqualizedOdds2]
  metrics['ConditionalUseAccuracyEquality'] = [ConditionalUseAccuracyEquality, ConditionalUseAccuracyEquality1 , ConditionalUseAccuracyEquality2]
  metrics['OverallAccuracyEquality'] = [OverallAccuracyEquality, OAE_discr, OAE_priv]
  metrics['TreatmentEquality'] = [TreatmentEquality, TreatmentEquality_discr, TreatmentEquality_priv]
  metrics['FORParity'] = [FORParity, FORParity_discr, FORParity_priv]
  metrics['FN'] = [FN_metric, FN_P_discr, FN_P_priv]
  metrics['FP'] = [FP_metric, FP_P_discr, FP_P_priv]

  for k in metrics.keys():
    value = standardization(rescale(metrics[k][0]))
    discr = metrics[k][1]
    priv = metrics[k][2]
    metrics[k] = {'Value': value, 'Discr_group': discr, 'Priv_group': priv}

  return metrics


# Fairness metrics computed using subtraction operator
def fairness_metrics_subtraction(confusion_matrix, threshold = 0.15):

  TP_priv, TN_priv, FP_priv, FN_priv, len_priv = retrieve_values(confusion_matrix['privileged'])
  TP_discr, TN_discr, FP_discr, FN_discr, len_discr = retrieve_values(confusion_matrix['discriminated'])

  GroupFairness_discr = (TP_discr+FP_discr)/len_discr
  GroupFairness_priv = (TP_priv+FP_priv)/len_priv
  GroupFairness = GroupFairness_discr-GroupFairness_priv

  if (TP_discr+FP_discr) == 0:
    PredictiveParity_discr = 0
  else:
    PredictiveParity_discr = (TP_discr)/(TP_discr+FP_discr)
  if (TP_priv+FP_priv) == 0:
    PredictiveParity_priv = 0
  else:
    PredictiveParity_priv = (TP_priv)/(TP_priv+FP_priv)
  PredictiveParity = PredictiveParity_discr-PredictiveParity_priv

  if TN_discr+FP_discr == 0:
    PredictiveEquality_discr = 0
  else:
    PredictiveEquality_discr = (FP_discr)/(TN_discr+FP_discr)
  if TN_priv+FP_priv == 0:
    PredictiveEquality_priv = 0
  else:
    PredictiveEquality_priv = (FP_priv)/(TN_priv+FP_priv)
  PredictiveEquality = PredictiveEquality_priv-PredictiveEquality_discr

  if TP_discr+FN_discr == 0:
    EqualOpportunity_discr = 0
  else:
    EqualOpportunity_discr = (FN_discr)/(TP_discr+FN_discr)
  if TP_priv+FN_priv == 0:
    EqualOpportunity_priv = 0
  else:
    EqualOpportunity_priv = (FN_priv)/(TP_priv+FN_priv)
  EqualOpportunity = EqualOpportunity_discr-EqualOpportunity_priv

  if FN_discr+TP_discr == 0:
    EqualizedOdds1 = 0
  elif FN_priv+TP_priv == 0:
    EqualizedOdds1 = 0
  else:
    EqualizedOdds1 = (TP_priv/(TP_priv+FN_priv))-(TP_discr/(TP_discr+FN_discr)) # (1-equalOpportunity_discr)/(1-equalOpportunity_priv)
  if FP_priv+TN_priv == 0:
    EqualizedOdds2 = 0
  elif FP_discr+TN_discr == 0:
    EqualizedOdds2 = 0
  else:
    EqualizedOdds2 = (FP_priv/(TN_priv+FP_priv))-(FP_discr/(TN_discr+FP_discr)) # = PredictiveEquality
  EqualizedOdds = sub_and_function(EqualizedOdds1, EqualizedOdds2, threshold)

  if TP_discr+FP_discr == 0:
    ConditionalUseAccuracyEquality1 = 0
  elif TP_priv+FP_priv == 0:
    ConditionalUseAccuracyEquality1 = 0
  else:
    ConditionalUseAccuracyEquality1 = (TP_priv/(TP_priv+FP_priv)) - (TP_discr/(TP_discr+FP_discr))
  if TN_discr+FN_discr == 0:
    ConditionalUseAccuracyEquality2 = 0
  elif TN_priv+FN_priv == 0:
    ConditionalUseAccuracyEquality2 = 0
  else:
    ConditionalUseAccuracyEquality2 = (TN_priv/(TN_priv+FN_priv)) - (TN_discr/(TN_discr+FN_discr))
  ConditionalUseAccuracyEquality = sub_and_function(ConditionalUseAccuracyEquality1, ConditionalUseAccuracyEquality2, threshold)

  OAE_discr = (TP_discr+TN_discr)/len_discr
  OAE_priv = (TP_priv+TN_priv)/len_priv
  OverallAccuracyEquality = OAE_discr-OAE_priv

  if FP_discr == 0:
    TreatmentEquality_discr = 0
  else:
    TreatmentEquality_discr = (FN_discr/FP_discr)
  if FP_priv == 0:
    TreatmentEquality_priv = 0
  else:
    TreatmentEquality_priv = (FN_priv/FP_priv)
  TreatmentEquality = TreatmentEquality_discr-TreatmentEquality_priv

  if TN_discr+FN_discr == 0:
    FORParity_discr = 0
  else:
    FORParity_discr = (FN_discr)/(TN_discr+FN_discr)
  if TN_priv+FN_priv == 0:
    FORParity_priv = 0
  else:
    FORParity_priv = (FN_priv)/(TN_priv+FN_priv)
  FORParity = FORParity_discr-FORParity_priv


  FN_P_discr =  (FN_discr)/len_discr
  FN_P_priv =  (FN_priv)/len_priv

  FP_P_discr = (FP_discr)/len_discr
  FP_P_priv =  (FP_priv)/len_priv

  #RecallParity = (TP_discr/(TP_discr+FN_discr))/(TP_priv/(TP_priv+FN_priv))

  metrics = {}
  metrics['GroupFairness'] = [GroupFairness, GroupFairness_discr, GroupFairness_priv]
  metrics['PredictiveParity'] = [PredictiveParity, PredictiveParity_discr, PredictiveParity_priv]
  metrics['PredictiveEquality'] = [PredictiveEquality, PredictiveEquality_discr, PredictiveEquality_priv]
  metrics['EqualOpportunity'] = [EqualOpportunity, EqualOpportunity_discr, EqualOpportunity_priv]
  metrics['EqualizedOdds'] = [EqualizedOdds, EqualizedOdds1, EqualizedOdds2]
  metrics['ConditionalUseAccuracyEquality'] = [ConditionalUseAccuracyEquality, ConditionalUseAccuracyEquality1 , ConditionalUseAccuracyEquality2]
  metrics['OverallAccuracyEquality'] = [OverallAccuracyEquality, OAE_discr, OAE_priv]
  metrics['TreatmentEquality'] = [TreatmentEquality, TreatmentEquality_discr, TreatmentEquality_priv]
  metrics['FORParity'] = [FORParity, FORParity_discr, FORParity_priv]
  metrics['FN'] = [FN_P_discr-FN_P_priv, FN_P_discr, FN_P_priv]
  metrics['FP'] = [FP_P_discr-FP_P_priv, FP_P_discr, FP_P_priv]

  for k in metrics.keys():
    value = standardization(metrics[k][0])
    discr = metrics[k][1]
    priv = metrics[k][2]
    metrics[k] = {'Value': value, 'Discr_group': discr, 'Priv_group': priv}

  return metrics

In [10]:
def compute_fairness_metrics(predictions_and_tests, target_variable_labels, models, n_splits):
  confusion_matrices = compute_confusion_matrices(predictions_and_tests, target_variable_labels, models, n_splits)
  fairness_metrics = {}
  sub_fairness_metrics = {}
  div_fairness_metrics = {}
  sub_dict = {}
  div_dict = {}
  #mitigation technique allow multiple models
  if mitigation not in without_model_mitigations:
    for model_name in (models):
      sub_dict = {}
      div_dict = {}
      for i in range(0,n_splits):
        model_split_conf_matrix = fairness_metrics_division(confusion_matrices[model_name][i])
        sub_dict[i] = fairness_metrics_subtraction(confusion_matrices[model_name][i])
        div_dict[i] = fairness_metrics_division(confusion_matrices[model_name][i])

      div_fairness_metrics[model_name] = div_dict
      sub_fairness_metrics[model_name] = sub_dict
  else:
    sub_dict = {}
    div_dict = {}
    for i in range(0,n_splits):
        sub_dict[i] = fairness_metrics_subtraction(confusion_matrices[i])
        div_dict[i] = fairness_metrics_division(confusion_matrices[i])

    div_fairness_metrics = div_dict
    sub_fairness_metrics = sub_dict

  fairness_metrics['division'] = div_fairness_metrics
  fairness_metrics['subtraction'] = sub_fairness_metrics

  return fairness_metrics

In [11]:
def compute_mean_std_dev_fairness_metrics(fairness_metrics, models):
  family_metrics = {}
  for f in family:
    model_metrics = {}
    #print(f)
    #mitigation technique allow multiple models
    if mitigation not in without_model_mitigations:
      for m in models:
        metric_dict = {}
        #print(m)
        for fair_m in fairness_catalogue:
          #print(fair_m)
          vec_metrics = []
          for i in range(0,n_splits):
            vec_metrics.append(fairness_metrics[f][m][i][fair_m]['Value'])
          #print(vec_metrics)
          #print(np.mean(vec_metrics), np.std(vec_metrics))
          metric_dict[fair_m] = [np.mean(vec_metrics), np.std(vec_metrics)]
        #print(metric_dict)
        model_metrics[m] = metric_dict
    #without multiple models
    else:
      metric_dict = {}
      for fair_m in fairness_catalogue:
        vec_metrics = []
        for i in range(0,n_splits):
          vec_metrics.append(fairness_metrics[f][i][fair_m]['Value'])
        metric_dict[fair_m] = [np.mean(vec_metrics), np.std(vec_metrics)]
      model_metrics = metric_dict

    family_metrics[f]=model_metrics

  return family_metrics

In [12]:
def compute_performance_metrics(predictions_and_tests, models, n_splits):
  accuracy, precision, recall, f1_score = compute_scores(predictions_and_tests, models, n_splits)

  if mitigation not in without_model_mitigations:
    #for each model compute mean and standard deviation
    acc = compute_mean_std_dev(accuracy, models)
    prec = compute_mean_std_dev(precision, models)
    rec = compute_mean_std_dev(recall, models)
    f1 = compute_mean_std_dev(f1_score, models)
  else:
    acc = compute_mean_std_dev(accuracy, None)
    prec = compute_mean_std_dev(precision, None)
    rec = compute_mean_std_dev(recall, None)
    f1 = compute_mean_std_dev(f1_score, None)

  performance_metrics = {}
  performance_metrics['accuracy'] = acc
  performance_metrics['precision'] = prec
  performance_metrics['recall'] = rec
  performance_metrics['f1_score'] = f1

  return performance_metrics

In [13]:
def compute_final_fairness_metrics(mitigation, predictions_and_tests, target_variable_labels, models, n_splits):
  if mitigation not in without_model_mitigations:
    fairness_metrics = compute_fairness_metrics(predictions_and_tests, target_variable_labels, models, n_splits)
    final_metrics = compute_mean_std_dev_fairness_metrics(fairness_metrics, models)
  else:
    fairness_metrics = compute_fairness_metrics(predictions_and_tests, target_variable_labels, None, n_splits)
    final_metrics = compute_mean_std_dev_fairness_metrics(fairness_metrics, None)

  return final_metrics

# Compute metrics

In [14]:
for sensible_attribute in sensible_attributes:
  for mitigation in all_mitigations:
    print(dataset_name, sensible_attribute, mitigation)
    # Load the correct source dataset, considering that pre-processing techniques
    # modify the original dataset, while in- and post- processing do not
    if mitigation in new_dataset_mitigations:
      dataset_path = path_to_project + '/data/mitigated/mitigated-{}-{}-{}.csv'.format(dataset_name, sensible_attribute, mitigation) if IN_COLAB else 'data/mitigated/mitigated-{}-{}.csv'.format(dataset_name, mitigation)
    else:
      dataset_path = path_to_project + '/data/preprocessed/preprocessed-{}.csv'.format(dataset_name) if IN_COLAB else 'data/preprocessed/preprocessed-{}.csv'.format(dataset_name)

    load_path = path_to_project + '/data/predictions_and_tests/pred_test-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
    with open(load_path, 'rb') as fp:
        predictions_and_tests = pickle.load(fp)

    df = pd.read_csv(dataset_path)
    #df = df.drop(columns=ignore_cols)
    #feature_cols = df.columns
    #sensible_values = [0, 1]  # 0 is the discriminated group, 1 the privileged one

    performance_metrics = compute_performance_metrics(predictions_and_tests, models, n_splits)
    #Save performance metrics
    save_path = path_to_project + '/measurements/performance_metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
    with open(save_path, 'wb') as fp:
        pickle.dump(performance_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)

    final_fairness_metrics = compute_final_fairness_metrics(mitigation, predictions_and_tests, target_variable_labels, models, n_splits)

    print("DIVISION")
    for metric in fairness_catalogue:
      print(metric)
      if mitigation in without_model_mitigations:
        print(final_fairness_metrics["division"][metric][0])
      else:
        for m in models:
          print(m, final_fairness_metrics["division"][m][metric][0])
      print('---------------')

    print("SUBTRACTION")
    for metric in fairness_catalogue:
      print(metric)
      if mitigation in without_model_mitigations:
        print(final_fairness_metrics["division"][metric][0])
      else:
        for m in models:
          print(m, final_fairness_metrics["subtraction"][m][metric][0])
      print('---------------')
    #Save the metrics results
    save_path = path_to_project + '/measurements/metrics-{}-{}-{}.p'.format(dataset_name, sensible_attribute, mitigation)
    with open(save_path, 'wb') as fp:
        pickle.dump(final_fairness_metrics, fp, protocol=pickle.HIGHEST_PROTOCOL)



diabetes-women AgeCategory fl-cr
DIVISION
GroupFairness
Logistic Regression 0.4485640250147889
Decision Tree 0.5024722885605
Bagging 0.15179711921572148
Random Forest 0.3905609422319231
Extremely Randomized Trees 0.47130541388729197
Ada Boost 0.47873060805214057
---------------
PredictiveParity
Logistic Regression 0.2543328944428689
Decision Tree 0.26523617643237546
Bagging 0.37119652090917504
Random Forest 0.2917774585091194
Extremely Randomized Trees 0.27282399238894284
Ada Boost 0.311978378190917
---------------
PredictiveEquality
Logistic Regression 0.4383333333333333
Decision Tree 0.7066369047619048
Bagging -0.2063973063973064
Random Forest 0.39825757575757575
Extremely Randomized Trees 0.5842365967365968
Ada Boost 0.3905438311688312
---------------
EqualOpportunity
Logistic Regression -0.8981180223285486
Decision Tree -0.5721902157992383
Bagging -0.24075187969924813
Random Forest -0.7736580086580087
Extremely Randomized Trees -0.8295815295815295
Ada Boost -0.5585212652844231
----

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

DIVISION
GroupFairness
Logistic Regression 0.13981781962642675
Decision Tree 0.07596316919444435
Bagging 0.08628104954845192
Random Forest 0.13503497996389413
Extremely Randomized Trees 0.12310301952167983
Ada Boost 0.0704783786165735
---------------
PredictiveParity
Logistic Regression 0.42473944914734396
Decision Tree 0.43834868143061423
Bagging 0.40925761957340906
Random Forest 0.3179128727364774
Extremely Randomized Trees 0.44027851938301765
Ada Boost 0.4250227127683822
---------------
PredictiveEquality
Logistic Regression -0.1962962962962963
Decision Tree -0.1820687134502924
Bagging -0.20799865643615645
Random Forest 0.11170830327080324
Extremely Randomized Trees -0.36875
Ada Boost -0.20392043686161337
---------------
EqualOpportunity
Logistic Regression 0.035354864433811806
Decision Tree 0.10494742266922716
Bagging -0.04027113237639557
Random Forest -0.010190287631303996
Extremely Randomized Trees 0.0392277663856611
Ada Boost 0.08407533986481351
---------------
EqualizedOdds
Log

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

DIVISION
GroupFairness
Logistic Regression 0.24758119064485787
Decision Tree 0.3087113838534866
Bagging 0.18695291538409814
Random Forest 0.35316813627949983
Extremely Randomized Trees 0.2798559837634541
Ada Boost 0.41456714802660616
---------------
PredictiveParity
Logistic Regression 0.3738042841642585
Decision Tree 0.33571367651955886
Bagging 0.3619921645115364
Random Forest 0.32156308140491746
Extremely Randomized Trees 0.3960948135794462
Ada Boost 0.3256352246749676
---------------
PredictiveEquality
Logistic Regression 0.07444444444444445
Decision Tree 0.29444444444444445
Bagging 0.04706293706293705
Random Forest 0.19174603174603175
Extremely Randomized Trees 0.1688888888888889
Ada Boost 0.4478306878306878
---------------
EqualOpportunity
Logistic Regression -0.09811802232854865
Decision Tree -0.07901830875891025
Bagging -0.06738095238095239
Random Forest -0.4381962481962482
Extremely Randomized Trees -0.04702380952380954
Ada Boost -0.446807701070859
---------------
EqualizedOdds